In [32]:
import pyspark
from pyspark.sql import SparkSession

# create the SparkSession locally with as many CPUs cores as possible
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

**Open the Spark UI at http://localhost:4040/**

In [33]:
# load in all green data into a Spark DataFrame
df_green = spark.read.parquet('./data/parquet/green/*/*')

# rename some columns via PySpark function to match up with yellow (will also be renamed)
df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

df_green.head()

Row(VendorID=2, pickup_datetime=datetime.datetime(2020, 1, 22, 13, 18, 32), dropoff_datetime=datetime.datetime(2020, 1, 22, 13, 45, 58), store_and_fwd_flag='N', RatecodeID=1, PULocationID=244, DOLocationID=41, passenger_count=1, trip_distance=5.22, fare_amount=22.0, extra=0.0, mta_tax=0.5, tip_amount=0.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=22.8, payment_type=1, trip_type=1, congestion_surcharge=0.0)

In [34]:
# load in all yellow data into a Spark DataFrame
df_yellow = spark.read.parquet('./data/parquet/yellow/*/*')

# rename some columns via PySpark function to match up with green
df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

df_yellow.head()

Row(VendorID=1, pickup_datetime=datetime.datetime(2020, 1, 1, 19, 45, 57), dropoff_datetime=datetime.datetime(2020, 1, 1, 20, 6, 13), passenger_count=1, trip_distance=6.1, RatecodeID=1, store_and_fwd_flag='N', PULocationID=162, DOLocationID=87, payment_type=1, fare_amount=21.0, extra=2.5, mta_tax=0.5, tip_amount=4.85, tolls_amount=0.0, improvement_surcharge=0.3, total_amount=29.15, congestion_surcharge=2.5)

**We want to eventually combine these two datasets into one large dataset**

In [35]:
print(df_green.count())
print(df_yellow.count())

2304517
39649199


In [36]:
# GET ALL COMMON COLUMNS BETWEEN THESE TWO DATASETS
common_columns = []

# get the set of unique columns from the yellow data
yellow_columns = set(df_yellow.columns)
print(yellow_columns)

{'PULocationID', 'DOLocationID', 'tip_amount', 'trip_distance', 'total_amount', 'pickup_datetime', 'congestion_surcharge', 'improvement_surcharge', 'VendorID', 'fare_amount', 'tolls_amount', 'passenger_count', 'dropoff_datetime', 'payment_type', 'mta_tax', 'store_and_fwd_flag', 'RatecodeID', 'extra'}


In [37]:
# get all columns from the yellow dataset that are also in the green dataset
#     and preserve order of cols
for col in df_green.columns:
    if col in yellow_columns:
        common_columns.append(col)
        
print(common_columns)

['VendorID', 'pickup_datetime', 'dropoff_datetime', 'store_and_fwd_flag', 'RatecodeID', 'PULocationID', 'DOLocationID', 'passenger_count', 'trip_distance', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'payment_type', 'congestion_surcharge']


In [38]:
from pyspark.sql import functions as F

**`.withColumn()` is a transformation function of a Spark DataFrame which is used to change the value, convert the datatype of an existing column, create a new column, and many more**
- https://sparkbyexamples.com/pyspark/pyspark-withcolumn/

**`pyspark.sql.functions.lit` is used to add a new column to DataFrame by assigning a literal or constant value**
- https://sparkbyexamples.com/pyspark/pyspark-lit-add-literal-constant/

In [39]:
# get all columns from green that are a part of the common columns
# and then add a new service_type column with value of 'green'
df_green_sel = df_green \
    .select(common_columns) \
    .withColumn('service_type', F.lit('green'))

# get all columns from yellow that are a part of the common columns
# and then add a new service_type column with value of 'yellow'
df_yellow_sel = df_yellow \
    .select(common_columns) \
    .withColumn('service_type', F.lit('yellow'))

In [41]:
# union the datasets together
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [42]:
# inspect the data
df_trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



In [43]:
# view the columns
df_trips_data.columns

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge',
 'service_type']

**In order to use Spark SQL, we have to tell Spark that our DataFrame is a table**

In [56]:
# tell Spark that our DataFrame is a table
# df_trips_data.registerTempTable('trips_data')  # deprecated
df_trips_data.createOrReplaceTempView('trips_data')

In [46]:
# do some Spark SQL on the data
spark.sql("""
SELECT
    service_type,
    count(1)
FROM
    trips_data
GROUP BY 
    service_type
""").show()

+------------+--------+
|service_type|count(1)|
+------------+--------+
|       green| 2304517|
|      yellow|39649199|
+------------+--------+



In [47]:
# save a SQL result to a NEW DataFrame, similar to our dbt models
# Get all revenue for each pickup zone/revenue location, for each service type, for each month
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

In [49]:
df_result.show(5)

+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+------------------------------------+--------------------------+------------------------+
|revenue_zone|      revenue_month|service_type|revenue_monthly_fare|revenue_monthly_extra|revenue_monthly_mta_tax|revenue_monthly_tip_amount|revenue_monthly_tolls_amount|revenue_monthly_improvement_surcharge|revenue_monthly_total_amount|revenue_monthly_congestion_surcharge|avg_montly_passenger_count|avg_montly_trip_distance|
+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+------------------------------------+--------------------------+------------------------+
|         218|2020-

In [50]:
df_result.head(5)

[Row(revenue_zone=218, revenue_month=datetime.datetime(2020, 1, 1, 0, 0), service_type='green', revenue_monthly_fare=24689.350000000126, revenue_monthly_extra=1561.75, revenue_monthly_mta_tax=121.5, revenue_monthly_tip_amount=18.7, revenue_monthly_tolls_amount=735.9000000000004, revenue_monthly_improvement_surcharge=246.0000000000036, revenue_monthly_total_amount=27375.149999999958, revenue_monthly_congestion_surcharge=0.0, avg_montly_passenger_count=1.0754716981132075, avg_montly_trip_distance=6.732052451539335),
 Row(revenue_zone=85, revenue_month=datetime.datetime(2020, 1, 1, 0, 0), service_type='green', revenue_monthly_fare=19874.88000000006, revenue_monthly_extra=1476.5, revenue_monthly_mta_tax=187.5, revenue_monthly_tip_amount=165.89999999999998, revenue_monthly_tolls_amount=354.6800000000002, revenue_monthly_improvement_surcharge=233.70000000000314, revenue_monthly_total_amount=22320.9099999999, revenue_monthly_congestion_surcharge=11.0, avg_montly_passenger_count=1.334975369458

**`.coalesce(*cols)` returns the first column that is not null and is *used to decrease the number of partitions in an efficient way***

**Can go to the Spark UI after running the next cell to see jobs/tasks running**

In [51]:
# use just 1 partition via .coalesce()
df_result.coalesce(1) \
    .write.parquet('data/report/revenue/', mode='overwrite')

In [54]:
# !dir data\report\revenue\